In [1]:
import os
os.chdir("../../")
os.getcwd()

'/home/yeonwoo/code/study/langchain_note'

In [2]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

### PromptTemplate

#### from_template() 메소드를 사용하여 PromptTemplate 객체 생성

In [4]:
from langchain_core.prompts import PromptTemplate
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성. format 메소드를 이용하여 변수에 값을 넣어줌
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
from langchain_openai import ChatOpenAI

In [7]:
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI()

chain = prompt | llm

In [8]:
chain.invoke("대한민국")

AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 24, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a7d369e0-4978-4c86-909e-a13c8d4fddf3-0', usage_metadata={'input_tokens': 24, 'output_tokens': 15, 'total_tokens': 39})

#### PromptTemplate 객체 생성과 동시에 prompt 생성

In [9]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"]
)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [10]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [13]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"
    }
)
prompt

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [14]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [15]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [16]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [17]:
chain = prompt_partial | llm
chain.invoke("대한민국")

AIMessage(content='대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 34, 'total_tokens': 66}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7f812472-74f6-4c43-85e0-50c5492ec856-0', usage_metadata={'input_tokens': 34, 'output_tokens': 32, 'total_tokens': 66})

In [18]:
chain.invoke({"country1": "대한민국", "country2": "호주"})

AIMessage(content='대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 31, 'total_tokens': 60}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ecf99ade-7ae9-4062-9307-c64da0e56838-0', usage_metadata={'input_tokens': 31, 'output_tokens': 29, 'total_tokens': 60})

#### 부분 변수 채움

In [19]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'July 16'

In [20]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [21]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)
prompt

PromptTemplate(input_variables=['n'], partial_variables={'today': <function get_today at 0x7fa6f2b0dab0>}, template='오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.')

In [22]:
prompt.format(n=3)

'오늘의 날짜는 July 16 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [23]:
chain = prompt | llm
chain.invoke(3).content

'1. Will Ferrell - 1967년 7월 16일\n2. Barry Sanders - 1968년 7월 16일\n3. Corey Feldman - 1971년 7월 16일'

In [24]:
chain.invoke({"today": "Jan 02", "n":3})

AIMessage(content='1. 김태희 (1980년 3월 29일)\n2. 조승우 (1980년 3월 28일)\n3. 배수지 (1994년 10월 10일)', response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 55, 'total_tokens': 114}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f49c5298-62a0-4c64-9c83-796e93de4cd8-0', usage_metadata={'input_tokens': 55, 'output_tokens': 59, 'total_tokens': 114})

### 파일로 부터 template 읽기

In [27]:
from langchain_core.prompts import load_prompt
prompt = load_prompt("prompt/prompt.json")
prompt

No `_type` key found, defaulting to `prompt`.


PromptTemplate(input_variables=['country1', 'country2'], template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [28]:
chain = prompt | llm
chain.invoke({"country1":"대한민국", "country2":"미국"})

AIMessage(content='대한민국의 수도는 서울이고, 미국의 수도는 워싱턴 D.C.입니다.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 34, 'total_tokens': 69}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5950ff4d-056c-483d-beaa-42f5a4620186-0', usage_metadata={'input_tokens': 34, 'output_tokens': 35, 'total_tokens': 69})

### ChatPromptTemplate
ChatPromptTemplate 은 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

메시지는 튜플(tuple) 형식으로 구성하며, (role, message) 로 구성하여 리스트로 생성할 수 있습니다.

In [29]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?'))])

In [30]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [31]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 당신의 이름은 {name} 입니다."),
        ("human", "안녕?"),
        ("ai", "안녕~ 무엇을 도와줄까?"),
        ("human", "{user_input}")
    ]
)
messages = chat_template.format_messages(
    name="cydchat", user_input="너의 이름은 뭐니?"
)

In [32]:
llm.invoke(messages).content

'제 이름은 cydchat 입니다. 필요하신 도움이 있으신가요?'

In [33]:
chain = chat_template | llm
chain.invoke({"name":"CYDchatbot", "user_input":"너의 이름은 무엇이니?"}).content

'제 이름은 CYDchatbot 입니다. 필요하신 도움이 있으면 언제든지 말씀해주세요!'

### MessagePlaceholder
LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 MessagePlaceholder 를 제공합니다.

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 요약 전문 AI 어시스턴트다. 너의 임무는 주요 키워드로 대화를 요약 하는 것이다."
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다.")
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='너는 요약 전문 AI 어시스턴트다. 너의 임무는 주요 키워드로 대화를 요약 하는 것이다.')), MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

In [35]:
formatted_chat_prompt  = chat_prompt.format(
    word_count = 5,
    conversation=[
        ("human","안녕하세요. 저는 돈을 저축하고 싶은데, 상품 하나만 추천해주세요."),
        ("ai", "저축 말고 펀드는 어떤가요? 위험부담은 적고, 수익은 많이 발생하는 펀드를 제가 하나 알고 있어요!")
    ]
)
print(formatted_chat_prompt)

System: 너는 요약 전문 AI 어시스턴트다. 너의 임무는 주요 키워드로 대화를 요약 하는 것이다.
Human: 안녕하세요. 저는 돈을 저축하고 싶은데, 상품 하나만 추천해주세요.
AI: 저축 말고 펀드는 어떤가요? 위험부담은 적고, 수익은 많이 발생하는 펀드를 제가 하나 알고 있어요!
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [42]:
prompt = chat_prompt.format_messages(
    word_count = 5,
    conversation=[
        ("human","안녕하세요. 저는 돈을 저축하고 싶은데, 상품 하나만 추천해주세요."),
        ("ai", "저축 말고 펀드는 어떤가요? 위험부담은 적고, 수익은 많이 발생하는 펀드를 제가 하나 알고 있어요!")
    ]
)

In [43]:
prompt

[SystemMessage(content='너는 요약 전문 AI 어시스턴트다. 너의 임무는 주요 키워드로 대화를 요약 하는 것이다.'),
 HumanMessage(content='안녕하세요. 저는 돈을 저축하고 싶은데, 상품 하나만 추천해주세요.'),
 AIMessage(content='저축 말고 펀드는 어떤가요? 위험부담은 적고, 수익은 많이 발생하는 펀드를 제가 하나 알고 있어요!'),
 HumanMessage(content='지금까지의 대화를 5 단어로 요약합니다.')]

In [45]:
llm.invoke(prompt)

AIMessage(content='저축을 원하면 펀드 추천.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 180, 'total_tokens': 197}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d01d0079-6454-47ac-91a5-f43d66a984eb-0', usage_metadata={'input_tokens': 180, 'output_tokens': 17, 'total_tokens': 197})

In [46]:
chain = chat_prompt | llm | StrOutputParser()

In [47]:
chain.invoke(
    {
        "word_count":5,
        "conversation":[
            ("human","안녕하세요. 저는 돈을 저축하고 싶은데, 상품 하나만 추천해주세요."),
            ("ai", "저축 말고 펀드는 어떤가요? 위험부담은 적고, 수익은 많이 발생하는 펀드를 제가 하나 알고 있어요!")
        ]
    }
)

'저축을 희망하는 사용자에게 펀드 추천.'